# Imports

In [143]:
import numpy as np
import pandas as pd
import swifter
# import pandas_profiling
import pprint
import pickle
import re

from datetime import datetime
import time

import pytz
from timezonefinder import TimezoneFinder

## Jupyter Configurations

In [144]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 500)

%matplotlib inline

## Function Definitions

In [145]:
def is_dst(dt=None, timezone="UTC"):
    if dt is None:
        dt = datetime.utcnow()
    timezone = pytz.timezone(timezone)
    timezone_aware_date = timezone.localize(dt, is_dst=None)
    return timezone_aware_date.tzinfo._dst.seconds != 0

def multipleReplace(text, wordDict):
    for key in wordDict:
        text = text.replace(key, wordDict[key])
    return text

## Read Data

In [146]:
%%time

df = pd.read_csv("US_Accidents_Dec19.csv")

print('Imported {:,} rows.'.format(len(df)))

Imported 2,974,335 rows.
CPU times: user 27.7 s, sys: 2.95 s, total: 30.6 s
Wall time: 31 s


# Data Cleaning

## Fill missing numeric values

In [147]:
%%time

df['Wind_Chill(F)'].fillna(df['Temperature(F)'], inplace=True)   #Went with Temp due to Wind_Chill.mean() giving values that didn't make sense.
df['Wind_Speed(mph)'].fillna(df['Wind_Speed(mph)'].mean(), inplace=True)

df['Precipitation(in)'].fillna(value=0, inplace=True)
df['Precipitation'] = df['Precipitation(in)'].apply(lambda x : True if float(x) > 0 else False)
df.drop('Precipitation(in)', axis = 1, inplace = True) # Converted to a bool named 'Precipitation'

CPU times: user 1.81 s, sys: 413 ms, total: 2.22 s
Wall time: 2.23 s


## Prepare the Datetime values

In [148]:
%%time

df['Start_Time'] = pd.to_datetime(df['Start_Time'])
df['End_Time'] = pd.to_datetime(df['End_Time'])
df['Weather_Timestamp'] = pd.to_datetime(df['Weather_Timestamp'])

df["Duration(s)"] = (df["End_Time"] - df["Start_Time"]) // pd.Timedelta('1s')
df['Start_Hour'] = df["Start_Time"].dt.hour
df['Start_Month'] = df["Start_Time"].dt.month
df['Start_Day'] = df["Start_Time"].dt.strftime("%A")

# Map Day name of week to number
d = {'Sunday': 1, 'Monday': 2, 'Tuesday': 3, 'Wednesday': 4, 'Thursday': 5, 'Friday': 6, 'Saturday': 7}

df['Start_Day'] = df['Start_Day'].map(d)

# Map Day name of week to number
d = {'Sunday': True, 'Monday': False, 'Tuesday': False, 'Wednesday': False, 'Thursday': False, 'Friday': False, 'Saturday': True}

df['Weekend'] = df['Start_Day'].map(d)

CPU times: user 20.8 s, sys: 981 ms, total: 21.8 s
Wall time: 22 s


## Fill missing timezones and futher refine

In [ ]:
%%time

# Get Timezone info for missing. Also converts the timezones to more localized ones from the standard four.
# May need to revisit and convert back to the standard four US timezones.
df['Timezone'] = df['Timezone'].astype('str')
tf = TimezoneFinder()
df['Timezone'] = df.swifter.apply(lambda row: tf.timezone_at(lat=row['Start_Lat'], lng=row['Start_Lng']),axis=1)

## Add Daylight Saving Time (DST) feature

In [ ]:
%%time

# Add Daylight Savings Time to allow for analysis.
df['DST'] = df.swifter.apply(lambda row: is_dst(row['Start_Time'], timezone=row['Timezone']), axis=1)

## Prepare the categorical data

In [149]:
%%time

df['Side'] = df['Side'].astype(str)
df.at[df[~df['Side'].isin(['L','R'])].index, 'Side'] = 'U'  #Replace unknown values with U
# df['Side'] = df['Side'].astype('category')


# df['Airport_Code'] = df['Airport_Code'].astype('category')
# df['Wind_Direction'] = df['Wind_Direction'].astype('category')
# df['Weather_Condition'] = df['Weather_Condition'].astype('category')


# Standarize the Wind_Direction values
wordDict = {'South': 'S', 'West': 'W', 'North': 'N', 'East': 'E', 'Variable': 'VAR', 'Calm': 'CALM'}
df['Wind_Direction'].fillna('VAR', inplace=True)
df['Wind_Direction'] = df.swifter.apply(lambda row: multipleReplace(row['Wind_Direction'], wordDict), axis = 1)


df['Weather_Condition'] = df['Weather_Condition'].replace(' \/ Windy', '', regex=True)

df['Weather_Condition'] = df['Weather_Condition'].replace('T-Storm', 'Thunderstorm', regex=True)
df['Weather_Condition'] = df['Weather_Condition'].replace('Rain with Thunder', 'Thunderstorm', regex=True)
df['Weather_Condition'] = df['Weather_Condition'].replace('Thunderstorms', 'Thunderstorm', regex=True)
df['Weather_Condition'] = df['Weather_Condition'].replace('^Thunder$', 'Thunderstorm', regex=True)

df['Weather_Condition'] = df['Weather_Condition'].replace('Mostly', '', regex=True)
df['Weather_Condition'] = df['Weather_Condition'].replace('Partly', '', regex=True)
df['Weather_Condition'] = df['Weather_Condition'].replace('Scattered', '', regex=True)
df['Weather_Condition'] = df['Weather_Condition'].replace('Light', '', regex=True)
df['Weather_Condition'] = df['Weather_Condition'].replace('Heavy', '', regex=True)
df['Weather_Condition'] = df['Weather_Condition'].replace('Patches of ', '', regex=True)
df['Weather_Condition'] = df['Weather_Condition'].replace(' and Rain', '', regex=True)
df['Weather_Condition'] = df['Weather_Condition'].replace(' in the Vicinity', '', regex=True)
df['Weather_Condition'] = df['Weather_Condition'].replace('Shallow', '', regex=True)
df['Weather_Condition'] = df['Weather_Condition'].replace('Widespread', '', regex=True)
df['Weather_Condition'] = df['Weather_Condition'].replace('Blowing ', '', regex=True)
df['Weather_Condition'] = df['Weather_Condition'].replace('Partial ', '', regex=True)
df['Weather_Condition'] = df['Weather_Condition'].replace('Freezing ', '', regex=True)
df['Weather_Condition'] = df['Weather_Condition'].replace('^Showers$', 'Rain', regex=True)
df['Weather_Condition'] = df['Weather_Condition'].replace(' Showers$', '', regex=True)
df['Weather_Condition'] = df['Weather_Condition'].replace(' Shower$', '', regex=True)
df['Weather_Condition'] = df['Weather_Condition'].replace('Drizzle', 'Rain', regex=True)
df['Weather_Condition'] = df['Weather_Condition'].replace(' Grains', '', regex=True)
df['Weather_Condition'] = df['Weather_Condition'].replace('Thunder and Hail', 'Hail', regex=True)
df['Weather_Condition'] = df['Weather_Condition'].replace('Thunderstorm with Small Hail', 'Hail', regex=True)
df['Weather_Condition'] = df['Weather_Condition'].replace('N/A Precipitation', 'None', regex=True)
df['Weather_Condition'] = df['Weather_Condition'].replace('with Thunder', '', regex=True)
df['Weather_Condition'] = df['Weather_Condition'].replace('Thunderstorm and', '', regex=True)
df['Weather_Condition'] = df['Weather_Condition'].replace('Small Hail', 'Hail', regex=True)
df['Weather_Condition'] = df['Weather_Condition'].replace('Low Drifting Snow', 'Snow', regex=True)
df['Weather_Condition'] = df['Weather_Condition'].replace('(\w+)\s\w+\s\w+$', r'\1', regex=True)

df['Weather_Condition'] = df['Weather_Condition'].replace(' Whirls', '', regex=True)


df['Weather_Condition'] = df['Weather_Condition'].replace('^(\w+)\s\/.+$', r'\1', regex=True)

df['Weather_Condition'] = df['Weather_Condition'].replace('^\s+', '', regex=True) #Remove leading spaces

CPU times: user 56.3 s, sys: 1.76 s, total: 58 s
Wall time: 58.9 s


In [150]:
%%time

# Cannot convert these before fillna occurs

df['Severity'] = df['Severity'].astype(int)

df['Zipcode'] = df['Zipcode'].str.split('-').str[0].astype(str)  # Throw away the Zip-4

CPU times: user 5.02 s, sys: 273 ms, total: 5.29 s
Wall time: 5.3 s


In [151]:
%%time

for col in ['Sunrise_Sunset','Civil_Twilight','Nautical_Twilight','Astronomical_Twilight']:
    df[col] = np.where(df[col].str.contains("Day"), True, False)

CPU times: user 6.81 s, sys: 521 ms, total: 7.33 s
Wall time: 7.34 s


In [152]:
%%time

# Drop unecessary columns

df.drop(['Source'], axis = 1, inplace = True) # Source doesn't seem to be useful as it's three values
df.drop(['End_Lat','End_Lng'], axis = 1, inplace = True) # Primarily NaN
df.drop(['Distance(mi)'], axis = 1, inplace = True) #Doesn't seem to be relevant as approx. 2M rows are 0.01

CPU times: user 2.06 s, sys: 868 ms, total: 2.93 s
Wall time: 2.94 s


## Write out data to Pickle

In [63]:
%%time

outfile = open('US_Accidents_Dec19', 'wb')
pickle.dump(df, outfile)
outfile.close()

CPU times: user 6.01 s, sys: 3.82 s, total: 9.84 s
Wall time: 11 s
